# Face_Detection using [https://realpython.com/face-recognition-with-python/]

In [1]:
!pip install face_recognition

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.1/100.1 MB 8.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for face-recognition-models: filename=face_recognition_models-0.3.0-py2.py3-none-any.whl size=100566170 sha256=43dbe215c48af7211f921dd64ff1fba563efcb9cdb3fcd51c752dd42af7d2272
  Stored in directory: /root/.cache/pip/wheels/7a/eb/cf/e9eced74122b679557f597bb7c8e4c739cfcac526db1fd523d
Successfully built face-recognition-models


In [2]:
import cv2

# for running on colab
# from google.colab.patches import cv2_imshow

from matplotlib import pyplot as plt
from PIL import Image, ImageDraw

import time

from collections import Counter

import pickle

import argparse

from pathlib import Path
import shutil
import os

import face_recognition

import json

## Step 1: Prepare Your Environment and Data

In [3]:
# mkdir face_recognizer

In [4]:
# cd face_recognizer

In [5]:
# mkdir output

In [6]:
# mkdir training

In [7]:
# mkdir validation

In [8]:
#@title Kaggle Credential { display-mode: "form" }

username = 'mraliamani' #@param {type: "string"}
api_key = '3d96c5a449b86f3bb0eae62a2832df3a' #@param {type: "string"}


if username and api_key:
    token = {"username": username, "key": api_key}

    !mkdir ~/.kaggle
    !mkdir /content/.kaggle
    with open('/content/.kaggle/kaggle.json', 'w') as f:
        json.dump(token, f)

    !cp /content/.kaggle/kaggle.json ~/.kaggle/kaggle.json
    !chmod 600 /root/.kaggle/kaggle.json

    print('Your are ready to use kaggle API!')

Your are ready to use kaggle API!


In [9]:
!kaggle datasets download -d adg1822/7-celebrity-images

 54% 12.0M/22.3M [00:00<00:00, 120MB/s]
100% 22.3M/22.3M [00:00<00:00, 162MB/s]


In [10]:
!unzip /content/7-celebrity-images.zip

Archive:  /content/7-celebrity-images.zip
  inflating: Newdata/train/ben_afflek/httpcsvkmeuaeccjpg.jpg  
  inflating: Newdata/train/ben_afflek/httpimagesfandangocomrImageRendererredesignstaticimgnoxportraitjpgpcpcpcimagesmasterrepositoryperformerimagespjpg.jpg  
  inflating: Newdata/train/ben_afflek/httpssmediacacheakpinimgcomxdbbdbbbececacdecdcdfjpg.jpg  
  inflating: Newdata/train/ben_afflek/httpssmediacacheakpinimgcomxdfdfadcfeabjpg.jpg  
  inflating: Newdata/train/ben_afflek/httpssmediacacheakpinimgcomxedaedabcbefbcbabbjpg.jpg  
  inflating: Newdata/train/ben_afflek/httpssmediacacheakpinimgcomxeebdfdbaaajpg.jpg  
  inflating: Newdata/train/ben_afflek/httpssmediacacheakpinimgcomxeeedcacddeccccacfjpg.jpg  
  inflating: Newdata/train/ben_afflek/httpsuploadwikimediaorgwikipediacommonsthumbddBenAffleckbyGageSkidmorejpgpxBenAffleckbyGageSkidmorejpg.jpg  
  inflating: Newdata/train/ben_afflek/httptrwebimgacstanetcxbdddmediasnmediajpg.jpg  
  inflating: Newdata/train/ben_afflek/httpwwwaces

In [11]:
def move_contents(source_folder, destination_folder):
  for filename in os.listdir(source_folder):
      src_file = os.path.join(source_folder, filename)
      dst_file = os.path.join(destination_folder, filename)

      shutil.move(src_file, dst_file)

In [12]:
move_contents("/content/Newdata/train", "/content/training")

In [13]:
move_contents("/content/Newdata/val", "/content/validation")

In [14]:
!mv /content/Newdata/unknown /content

In [15]:
!rm -rf /content/7-celebrity-images.zip

In [16]:
!rm -rf /content/Newdata

## Step 2: Load Training Data and Train Your Model

HOG (histogram of oriented gradients) is a common technique for object detection. For this tutorial, you only need to remember that it works best with a CPU.
CNN (convolutional neural network) is another technique for object detection. In contrast to a HOG, a CNN works better on a GPU, otherwise known as a video card.

In [17]:
DEFAULT_ENCODINGS_PATH = Path('output/encodings.pkl')
BOUNDING_BOX_COLOR = 'blue'
TEXT_COLOT = 'white'

In [18]:
parser = argparse.ArgumentParser(description = 'Recognize faces in an image')
parser.add_argument('--train', action='store_true', help='train on input data')
parser.add_argument('--validate', action='store_true', help='validate trained model')
parser.add_argument('--test', action='store_true', help='test the model with an unknown image')
parser.add_argument(
    '-m',
    action='store',
    default='hog',
    choices=['hog', 'cnn'],
    help='Which model to use for training: hog (CPU), cnn (GPU)'
)
parser.add_argument('-f', action='store', help='Path to an image with an unknown face')
args = parser.parse_args()

In [19]:
Path('training').mkdir(exist_ok=True)
Path('output').mkdir(exist_ok=True)
Path('validation').mkdir(exist_ok=True)

In [20]:
def encode_known_faces(model: str = 'hog', encodings_location: Path = DEFAULT_ENCODINGS_PATH) -> None:
    names = []
    encodings = []

    for filepath in Path('training').glob('*/*'):
        name  = filepath.parent.name
        image = face_recognition.load_image_file(filepath)

        # bounding box
        face_locations = face_recognition.face_locations(image, model = model)
        face_encodings = face_recognition.face_encodings(image, face_locations)

        for encoding in face_encodings:
            names.append(name)
            encodings.append(encoding)

    name_encodings = {'names': names, 'encodings': encodings}
    with encodings_location.open(mode='wb') as f:
        pickle.dump(name_encodings, f)

In [27]:
# encode_known_faces(model='cnn')

In [28]:
# with open('output/encodings.pkl', mode='rb') as f:
#     loaded_encodings = pickle.load(f)
# print(loaded_encodings['names'])

['madonna', 'madonna', 'madonna', 'madonna', 'madonna', 'madonna', 'madonna', 'madonna', 'madonna', 'madonna', 'madonna', 'madonna', 'madonna', 'madonna', 'madonna', 'jerry_seinfeld', 'jerry_seinfeld', 'jerry_seinfeld', 'jerry_seinfeld', 'jerry_seinfeld', 'jerry_seinfeld', 'jerry_seinfeld', 'jerry_seinfeld', 'jerry_seinfeld', 'jerry_seinfeld', 'jerry_seinfeld', 'jerry_seinfeld', 'jerry_seinfeld', 'jerry_seinfeld', 'jerry_seinfeld', 'jobiden', 'jobiden', 'jobiden', 'jobiden', 'jobiden', 'jobiden', 'jobiden', 'jobiden', 'jobiden', 'jobiden', 'jobiden', 'jobiden', 'jobiden', 'jobiden', 'jobiden', 'ben_afflek', 'ben_afflek', 'ben_afflek', 'ben_afflek', 'ben_afflek', 'ben_afflek', 'ben_afflek', 'ben_afflek', 'ben_afflek', 'ben_afflek', 'ben_afflek', 'ben_afflek', 'ben_afflek', 'elon_musk', 'elon_musk', 'elon_musk', 'elon_musk', 'elon_musk', 'elon_musk', 'elon_musk', 'elon_musk', 'elon_musk', 'elon_musk', 'elon_musk', 'elon_musk', 'elon_musk', 'elon_musk', 'elon_musk', 'elon_musk', 'elton_jo

## Step 3: Recognize Unlabeled Faces

In [23]:
def recognize_faces(
    image_location: str,
    model: str = 'hog',
    encodings_location: Path = DEFAULT_ENCODINGS_PATH
) -> None:
    with encodings_location.open(mode = 'rb') as f:
        loaded_encodings = pickle.load(f)

    input_image = face_recognition.load_image_file(image_location)

    input_face_locations = face_recognition.face_locations(input_image, model=model)
    input_face_encodings = face_recognition.face_encodings(input_image, input_face_locations)

    pillow_image = Image.fromarray(input_image)
    draw = ImageDraw.Draw(pillow_image)

    for bounding_box, unknown_encoding in zip(
        input_face_locations, input_face_encodings
    ):
        name = _recognize_face(unknown_encoding, loaded_encodings)
        if not name:
            name = 'Unknown'
        print(name, bounding_box)
        _display_face(draw, bounding_box, name)

    del draw
    pillow_image.show()

In [24]:
def _recognize_face(unknown_encoding, loaded_encodings):
    boolean_matches = face_recognition.compare_faces(
        loaded_encodings['encodings'], unknown_encoding
    )
    votes = Counter(
        name
        for match, name in zip(boolean_matches, loaded_encodings['names'])
    )
    if votes:
        return votes.most_common(1)[0][0]

## Step 4: Display Results

In [25]:
def _display_face(draw, bounding_box, name):
    top, right, bottom, left = bounding_box
    draw.rectangle(((left, top), (right, bottom)), outline = BOUNDING_BOX_COLOR)
    text_left, text_top, text_right, text_bottom = draw.textbbox(
        (left, bottom), name
    )
    draw.rectangle(
        ((text_left, text_top), (text_right, text_bottom)),
        fill='blue',
        outline='blue'
    )
    draw.text((text_left, text_top), name, fill='white')

In [34]:
# test_images
recognize_faces('/content/unknown/anand.jpg', model='cnn')

elon_musk (1347, 1680, 1854, 1173)


In [35]:
recognize_faces('/content/unknown/hillary-clinton-9251306-2-402.jpg', model='cnn')

elon_musk (177, 422, 470, 128)


## Step 5: Validate Your Model

In [29]:
def validate(model: str = 'hog'):
    for filepath in Path('validation').rglob('*'):
        if filepath.is_file():
            recognize_faces(image_location = str(filepath.absolute()), model = model)

In [30]:
# validate(model='cnn)

## Step 6: Add Command-Line Arguments

In [31]:
if __name__ == '__main__':
    if args.train:
        encode_known_faces(args.m)
    if args.validate:
        validate(args.m)
    if args.test:
        recognize_faces(image_location=args.f, model=args.m)


## Step 7: Perform Face Recognition With Python